In [12]:
import os

import bk.load
import bk.compute
import bk.plot
import bk.signal

import numpy as np
import pandas as pd

import scipy.signal
import scipy.stats
import scipy.io

import matplotlib.pyplot as plt
import matplotlib as mpl
import neuroseries as nts


import itertools
from tqdm import tqdm



In [13]:
def training_intervals():
    run_ = scipy.io.loadmat('runintervals.mat')['runintervals']
    runintervals = nts.IntervalSet(run_[:,0],run_[:,1],time_units='s')
    return runintervals

In [14]:
def mean_firing(neurons,intervals):
    binSize = 0.5
    t,fr = bk.compute.binSpikes(neurons,binSize)
    t = nts.Ts(t,time_units='s')
    fr = np.mean(fr[:,intervals.in_interval(t)>= 0]/binSize,0)
    t = t[intervals.in_interval(t)>=0]
    fr = nts.Tsd(t.index.values, d = fr)
    fr = bk.compute.nts_smooth(fr,1000,1)
    return fr

In [15]:
COLOR = 'white'
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['axes.edgecolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR
mpl.rcParams['figure.facecolor'] = '#282C34'
mpl.rcParams['axes.facecolor'] = '#282C34'

In [58]:
path = '/media/billel/GGLab-01-BK/DATA/Rat08-20130713/'
os.chdir(path)
bk.load.path = path
bk.load.rat = 8
bk.load.session = 'Rat08-20130713'
bk.load.day = 6

In [22]:
bk.load.current_session()
neurons,metadata = bk.load.loadSpikeData(bk.load.path)
states = bk.load.states()
pos = bk.load.pos()
train = training_intervals()
pos = pos.restrict(train)
speed = bk.compute.old_speed(pos,0.5)
speed = speed[speed.values>1]

Rat : 8 on day : 6
Working with session Rat08-20130713 @ Z:\Rat08\Rat08-20130713
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy


In [23]:
import sys
eps = sys.float_info.epsilon

In [28]:
log = True
%matplotlib qt
stru = ['Hpc','BLA']
types = ['Pyr','Int']

for s,t in itertools.product(stru,types):
    print(s,t)
    neur = neurons[(metadata.Region == s) & (metadata.Type == t)]
    meta = metadata[(metadata.Region == s) & (metadata.Type == t)]
    for n,shank,id in zip(neur,meta['Shank'],meta['Id']):
        fr = mean_firing([n],train)
        stats = (scipy.stats.linregress(speed.realign(fr).values,fr.values))
        if (stats.pvalue <0.05) & (stats.rvalue > 0.2):
            if log:
                fr.values = np.log(fr.values+eps)
                speed.values = np.log(speed.values +eps)
            print(stats)
            plt.figure()
            plt.scatter(speed.realign(fr).values,fr.values,1)
            x = np.linspace(0,40,2500)
            y = stats.intercept + x*stats.slope
            plt.plot(x,y,'red')
            plt.title(s+t+' '+str(shank)+' '+str(id))
            print(scipy.stats.linregress(speed.realign(fr).values,fr.values+eps))
            plt.ylabel('FiringRates')
            plt.xlabel('Speed(cm/s)')
            i+=1
# plt.legend(list(itertools.product(stru,types)))

# plt.subplot(2,1,2,sharex = sub)
# plt.plot(speed)

Hpc Pyr


AttributeError: can't set attribute

In [11]:
plt.show()

In [ ]:
plt.ion()

In [16]:
bk.load.current_session('Z:\\Rat08\\Rat08-20130710')
neurons,metadata = bk.load.loadSpikeData(bk.load.path)

for i in np.unique(metadata['Shank']):
    print(i,len(neurons[metadata['Shank']==i]))

FileNotFoundError: [Errno 2] No such file or directory: 'Z:/All-Rats/Billel/session_indexing.csv'

In [287]:
a = np.array([np.nan])

AttributeError: 'numpy.ndarray' object has no attribute 'dropna'